In [11]:
import arcpy
import psycopg2


In [ ]:
#create list of coordinates

coordinates = [ (0,0), (-10,0), (-10,10), (-5,15), (0,10)]

points = [arcpy.Point(*coords) for coords in coordinates] #iterate through list to make the coords into acrpy points

In [ ]:
#create array from the points - necessary for larger geometry
array = arcpy.Array(points)

#create polygon from array
polygon = arcpy.Polygon(array)

print(polygon)

<geoprocessing describe geometry object object at 0x000002295433A100>


In [ ]:
# Convert my polygon to WKT
wkt = polygon.WKT
print(wkt)

MULTIPOLYGON (((0 0, 0 10, -5 15, -10 10, -10 0, 0 0)))


In [ ]:
#set connection to database - credentials removed for submission
connection = psycopg2.connect(
    dbname= ,
    user= 
    password= ,
    host= ,
    port= 
)

cursor = connection.cursor()

# Check if the table exists and create it if it doesn't
create_table_query = """
CREATE TABLE IF NOT EXISTS my_polygon (
    id SERIAL PRIMARY KEY,
    geom GEOMETRY
);
"""
cursor.execute(create_table_query)

# Insert the polygon WKT into the table
insert_query = """
INSERT INTO my_polygon (geom)
VALUES (ST_GeomFromText(%s, 4326))
"""
cursor.execute(insert_query, (wkt,))
connection.commit()

cursor.close()
connection.close() #need to close cursor and the connection 